# Adversarial machine learning

В самом широком понимании этого слова, это наука об атаках на алгоритмы машинного обучения и способах защиты от них

В данной работе будет рассмотрен пример такой атаки. Сначала будет обучена модель для распознавания цифр на картинке, затем будут сформированы примеры (adversarial examples), изменяющие показания модели

Работа выполнена студентом БИБ233 МИЭМ НИУ ВШЭ Коноваловым Матвеем



## Обучение модели
Сначала скачаем [модель](https://github.com/ultralytics/ultralytics/blob/main/docs/en/models/yolov8.md)

Была выбрана модель YOLOv8, предназначенная для задачи классификации, имеющая малое количество параметров. Большие модели могли бы замедлить эксперимент, меньшие представляли бы меньше интереса

In [12]:
from ultralytics import YOLO

model = YOLO("yolov8s-cls.pt")
model.info()

YOLOv8s-cls summary: 56 layers, 6,361,736 parameters, 0 gradients, 13.6 GFLOPs


(56, 6361736, 0, 13.609779199999998)

Затем скачаем датасет и приведем его в формат, совместимый с YOLO

In [10]:
from datasets import load_dataset
from pathlib import Path

ds = load_dataset("ufldl-stanford/svhn", "cropped_digits")
root = Path("datasets/svhn_cls")
splits = ["train", "test"]

for split in splits:
    split_ds = ds[split]
    for i, example in enumerate(split_ds):
        label = str(example["label"])
        out_dir = root / split / label
        out_dir.mkdir(parents=True, exist_ok=True)
        example["image"].save(out_dir / f"{i}.png")

Обучим модель

In [14]:
model.train(data=root,
            epochs=50,
            batch=128)
model.val()

Ultralytics 8.3.153  Python-3.11.9 torch-2.1.2+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 8188MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=128, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=datasets\svhn_cls, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, 

train: Scanning C:\Users\ok__\PycharmProjects\MLSecOps\datasets\svhn_cls\train... 73257 images, 0 corrupt: 100%|██████████| 73257/73257 [00:00<?, ?it/s]


val: Fast image access  (ping: 0.20.0 ms, read: 5.31.1 MB/s, size: 1.7 KB)


val: Scanning C:\Users\ok__\PycharmProjects\MLSecOps\datasets\svhn_cls\test... 26032 images, 0 corrupt: 100%|██████████| 26032/26032 [00:00<?, ?it/s]


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 26 weight(decay=0.0), 27 weight(decay=0.001), 27 bias(decay=0.0)
Image sizes 224 train, 224 val
Using 8 dataloader workers
Logging results to runs\classify\train3
Starting training for 50 epochs...

      Epoch    GPU_mem       loss  Instances       Size


...

               classes   top1_acc   top5_acc: 100%|██████████| 102/102 [00:09<00:00, 10.24it/s]


                   all       0.97      0.997
Speed: 0.1ms preprocess, 0.2ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs\classify\train3
Ultralytics 8.3.153  Python-3.11.9 torch-2.1.2+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 8188MiB)
YOLOv8s-cls summary (fused): 30 layers, 5,088,010 parameters, 0 gradients, 12.5 GFLOPs
WARNING Dataset 'split=val' not found, using 'split=test' instead.
train: C:\Users\ok__\PycharmProjects\MLSecOps\datasets\svhn_cls\train... found 73257 images in 10 classes  
val: C:\Users\ok__\PycharmProjects\MLSecOps\datasets\svhn_cls\test... found 26032 images in 10 classes  
test: C:\Users\ok__\PycharmProjects\MLSecOps\datasets\svhn_cls\test... found 26032 images in 10 classes  
val: Fast image access  (ping: 0.00.0 ms, read: 19.03.7 MB/s, size: 1.7 KB)


val: Scanning C:\Users\ok__\PycharmProjects\MLSecOps\datasets\svhn_cls\test... 26032 images, 0 corrupt: 100%|██████████| 26032/26032 [00:00<?, ?it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 204/204 [00:12<00:00, 16.31it/s]


                   all       0.97      0.997
Speed: 0.1ms preprocess, 0.3ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs\classify\train32


ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000016839F93AD0>
curves: []
curves_results: []
fitness: 0.9836931526660919
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.9702289700508118, 'metrics/accuracy_top5': 0.9971573352813721, 'fitness': 0.9836931526660919}
save_dir: WindowsPath('runs/classify/train32')
speed: {'preprocess': 0.05283406576525764, 'inference': 0.30222762753539256, 'loss': 8.201060240060639e-05, 'postprocess': 0.0003986708666821197}
task: 'classify'
top1: 0.9702289700508118
top5: 0.9971573352813721

Отлично, модель обучилась и имеет долю правильных ответов 97%, при этом правильный ответ находится в топ 5 в 99,7% случаев
Обученную модель можно скачать по [ссылке](https://disk.360.yandex.ru/d/r-5SqYBdwTfuuQ) (рекомендуется положить её в папку models)



## Проведение атаки

Пусть наша [модель](https://disk.360.yandex.ru/d/r-5SqYBdwTfuuQ) находится в папке models. Считаем её

In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8s-cls.pt")